# AWS Lambda event['body'] Inspection
I was getting a DataFrame constructor error. I updated the AWS Lambda function to pickle the `event` object and upload it to S3. This workbook inspects the object and helped identify the steps required to convert the event['body'] object into a pandas.DataFrame.

## Resolution
The default `dataframe.to_json` method that was initially used is difficult to parse. I added the `orient='split'` argument, which yielded a more user-friendly format. One strange feature is that `json.loads` needs to be called twice in order to convert the string object into a json object.

In [1]:
import boto3
import pickle
import os

session = boto3.Session(aws_access_key_id=os.getenv('AWS_ADMIN_ACCESS'),
                        aws_secret_access_key=os.getenv('AWS_ADMIN_SECRET'))

s3 = session.client('s3')

bucket = 'gwilson253awsprojects'
key = 'neptune/event_body.pkl'

obj = s3.get_object(Bucket=bucket, Key=key)

event_body = pickle.loads(obj['Body'].read())
event_body

'"{\\"columns\\":[\\"alcohol\\",\\"malic_acid\\",\\"ash\\",\\"alcalinity_of_ash\\",\\"magnesium\\",\\"total_phenols\\",\\"flavanoids\\",\\"nonflavanoid_phenols\\",\\"proanthocyanins\\",\\"color_intensity\\",\\"hue\\",\\"od280\\\\/od315_of_diluted_wines\\",\\"proline\\"],\\"index\\":[129,1,16,130,149,90,58,111,122,97],\\"data\\":[[12.04,4.3,2.38,22.0,80.0,2.1,1.75,0.42,1.35,2.6,0.79,2.57,580.0],[13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0],[14.3,1.92,2.72,20.0,120.0,2.8,3.14,0.33,1.97,6.2,1.07,2.65,1280.0],[12.86,1.35,2.32,18.0,122.0,1.51,1.25,0.21,0.94,4.1,0.76,1.29,630.0],[13.08,3.9,2.36,21.5,113.0,1.41,1.39,0.34,1.14,9.4,0.57,1.33,550.0],[12.08,1.83,2.32,18.5,81.0,1.6,1.5,0.52,1.64,2.4,1.08,2.27,480.0],[13.72,1.43,2.5,16.7,108.0,3.4,3.67,0.19,2.04,6.8,0.89,2.87,1285.0],[12.52,2.43,2.17,21.0,88.0,2.55,2.27,0.26,1.22,2.0,0.9,2.78,325.0],[12.42,4.43,2.73,26.5,102.0,2.2,2.13,0.43,1.71,2.08,0.92,3.12,365.0],[12.29,1.41,1.98,16.0,85.0,2.55,2.5,0.29,1.77,2.9,1.23,2.74

In [2]:
from json import loads
j = loads(loads(event_body))
j['columns']

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [3]:
from pandas import DataFrame
DataFrame(data=j['data'], columns=j['columns'], index=j['index'])

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
129,12.04,4.30,2.38,22.0,80.0,2.10,1.75,0.42,1.35,2.60,0.79,2.57,580.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
16,14.30,1.92,2.72,20.0,120.0,2.80,3.14,0.33,1.97,6.20,1.07,2.65,1280.0
130,12.86,1.35,2.32,18.0,122.0,1.51,1.25,0.21,0.94,4.10,0.76,1.29,630.0
149,13.08,3.90,2.36,21.5,113.0,1.41,1.39,0.34,1.14,9.40,0.57,1.33,550.0
90,12.08,1.83,2.32,18.5,81.0,1.60,1.50,0.52,1.64,2.40,1.08,2.27,480.0
58,13.72,1.43,2.50,16.7,108.0,3.40,3.67,0.19,2.04,6.80,0.89,2.87,1285.0
111,12.52,2.43,2.17,21.0,88.0,2.55,2.27,0.26,1.22,2.00,0.90,2.78,325.0
122,12.42,4.43,2.73,26.5,102.0,2.20,2.13,0.43,1.71,2.08,0.92,3.12,365.0
97,12.29,1.41,1.98,16.0,85.0,2.55,2.50,0.29,1.77,2.90,1.23,2.74,428.0
